In [1]:
%pip install lxml
%pip install nltk


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from lxml import etree
from pathlib import Path

In [3]:
files = list(Path("tlg0012").glob("./**/*perseus-eng*.xml"))

for f in files:
    print(f)

tlg0012/tlg001/tlg0012.tlg001.perseus-eng3.xml
tlg0012/tlg001/tlg0012.tlg001.perseus-eng4.xml
tlg0012/tlg003/tlg0012.tlg003.perseus-eng1.xml
tlg0012/tlg002/tlg0012.tlg002.perseus-eng4.xml
tlg0012/tlg002/tlg0012.tlg002.perseus-eng3.xml


In [4]:
TEI_NS = "http://www.tei-c.org/ns/1.0"
XML_NS = "http://www.w3.org/XML/1998/namespace"

NAMESPACES = {
    "tei": TEI_NS,
    "xml": XML_NS
}

In [5]:
for file in files:
    print(file)
    tree = etree.parse(file)
    text = tree.xpath(f"//tei:div[@subtype='card']//text()", namespaces=NAMESPACES)
    
    cleaned_text = []
    for t in text:
        if t.strip() != "":
            cleaned_text.append(t)

    if len(cleaned_text) > 0:
        with open(str(file).split("/")[-1].replace(".xml", ".txt"), "w+") as f:
            f.write('\n'.join(cleaned_text))



tlg0012/tlg001/tlg0012.tlg001.perseus-eng3.xml


tlg0012/tlg001/tlg0012.tlg001.perseus-eng4.xml
tlg0012/tlg003/tlg0012.tlg003.perseus-eng1.xml
tlg0012/tlg002/tlg0012.tlg002.perseus-eng4.xml
tlg0012/tlg002/tlg0012.tlg002.perseus-eng3.xml


In [6]:
import nltk

nltk.download("punkt")
nltk.download("punkt_tab")

[nltk_data] Downloading package punkt to /home/codespace/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [7]:
from nltk.tokenize import word_tokenize

tokenized_texts = {}

text_files = Path(".").glob("tlg0012.tlg00*.perseus-eng[1-4].txt")

for file in text_files:
    name = str(file)

    with open(file) as f:
        text = f.read().lower()
        tokens = word_tokenize(text)
        print(f"There are {len(tokens)} tokens in {name}.")
        
        tokenized_texts[name] = tokens


There are 200630 tokens in tlg0012.tlg001.perseus-eng3.txt.
There are 135463 tokens in tlg0012.tlg002.perseus-eng4.txt.
There are 175611 tokens in tlg0012.tlg001.perseus-eng4.txt.
There are 152631 tokens in tlg0012.tlg002.perseus-eng3.txt.


In [8]:
from collections import Counter

for filename, tokens in tokenized_texts.items():
    counts = Counter(tokens)

    tokenized_texts[filename] = {"tokens": tokens, "counts": counts}


In [9]:
tokenized_texts["tlg0012.tlg001.perseus-eng3.txt"]["counts"]["odysseus"]

128

In [10]:
df_achilles = 0
df_odysseus = 0

for filename, values in tokenized_texts.items():
    if "odysseus" in values['counts']:
        df_odysseus += 1
    
    if "achilles" in values["counts"]:
        df_achilles += 1

from math import log10

n_docs = len(tokenized_texts.keys())

idf_achilles = log10(n_docs / df_achilles)
idf_odysseus = log10(n_docs / df_odysseus)

print(idf_achilles)

0.0


In [11]:
for filename, values in tokenized_texts.items():
    total_terms = len(values['tokens'])

    tf_achilles = values['counts']['achilles'] / total_terms
    tf_odysseus = values['counts']['odysseus'] / total_terms

    tf_idf_achilles = tf_achilles * idf_achilles
    tf_idf_odysseus = tf_odysseus * idf_odysseus

    print(f"""In {filename}:
TF of achilles: {tf_achilles}
TF of odysseus: {tf_odysseus}
TF-IDF of achilles: {tf_idf_achilles}
TF-IDF of odysseus: {tf_idf_odysseus}
""")

In tlg0012.tlg001.perseus-eng3.txt:
TF of achilles: 0.002043562777251657
TF of odysseus: 0.000637990330459054
TF-IDF of achilles: 0.0
TF-IDF of odysseus: 0.0

In tlg0012.tlg002.perseus-eng4.txt:
TF of achilles: 0.0001254955227626732
TF of odysseus: 0.0042816119530794386
TF-IDF of achilles: 0.0
TF-IDF of odysseus: 0.0

In tlg0012.tlg001.perseus-eng4.txt:
TF of achilles: 0.002403038534032606
TF of odysseus: 0.0007061061095261686
TF-IDF of achilles: 0.0
TF-IDF of odysseus: 0.0

In tlg0012.tlg002.perseus-eng3.txt:
TF of achilles: 0.0001048279838302835
TF of odysseus: 0.0041603606082643765
TF-IDF of achilles: 0.0
TF-IDF of odysseus: 0.0



In [12]:
my_list = [1, 1, 2, 3, 3]

set(my_list)

{1, 2, 3}

In [13]:
non_universal_terms = {}

for filename, values in tokenized_texts.items():
    my_set = set(values['counts'].keys())

    for other_file, other_values in tokenized_texts.items():
        if other_file != filename:
            my_set -= set(other_values['counts'].keys())
    
    non_universal_terms[filename] = my_set

non_universal_terms

{'tlg0012.tlg001.perseus-eng3.txt': {'shallow',
  'heaped-up',
  'division',
  'pheneos',
  'declaring',
  'midmost',
  'burgeoneth',
  'quick-glancing',
  'wandereth',
  'grievously',
  'woodman',
  'serves',
  'deïphobus',
  'clonius',
  'skyward',
  'euneos',
  'bethinking',
  'people-devouring',
  'overweeningly',
  'promptings',
  'me—night',
  'adrastus',
  'neeks',
  '203.1',
  'fencing',
  'marsh',
  'bethougnt',
  '—thou',
  'hyllus',
  '521.1',
  '275.1',
  'down—lest',
  'couching-places',
  'antron',
  'gazeth',
  'stone—a',
  'presseth',
  'bosom—eurynome',
  'pedaeus',
  'perisheth',
  'brake',
  'inclineth',
  'apportioned',
  'anemoreia',
  'dulled',
  'younglings',
  'joinings',
  'balius',
  'send—even',
  'dallied',
  'leïtus',
  'formeth',
  'thwarted',
  'pen',
  'unfinished',
  'swelleth',
  'plaything',
  'ialmenus',
  'imbrasus',
  'phrygian',
  'clasping',
  'avow',
  'giveth',
  'cymodoce',
  'refuseth',
  'life—even',
  'echepolus',
  'railings',
  'taslets',

In [14]:
idf_non_universal = log10(n_docs / 1)
print(idf_non_universal)
tf_idf_non_universal = {}

for filename, values in tokenized_texts.items():
    total_terms = len(values['tokens']) 
    tf_idf_non_universal[filename] = {}

    for word in non_universal_terms[filename]: 
        tf = values['counts'][word] / total_terms 
        tf_idf = tf * idf_non_universal 

        tf_idf_non_universal[filename][word] = tf_idf

print(tf_idf_non_universal)

0.6020599913279624
{'tlg0012.tlg001.perseus-eng3.txt': {'shallow': 3.0008472876836084e-06, 'heaped-up': 1.2003389150734433e-05, 'division': 6.001694575367217e-06, 'pheneos': 3.0008472876836084e-06, 'declaring': 9.002541863050826e-06, 'midmost': 6.001694575367217e-06, 'burgeoneth': 3.0008472876836084e-06, 'quick-glancing': 3.0008472876836084e-06, 'wandereth': 3.0008472876836084e-06, 'grievously': 6.001694575367217e-06, 'woodman': 6.001694575367217e-06, 'serves': 3.0008472876836084e-06, 'deïphobus': 5.401525117830495e-05, 'clonius': 6.001694575367217e-06, 'skyward': 3.0008472876836084e-06, 'euneos': 3.0008472876836084e-06, 'bethinking': 9.002541863050826e-06, 'people-devouring': 3.0008472876836084e-06, 'overweeningly': 6.001694575367217e-06, 'promptings': 6.001694575367217e-06, 'me—night': 3.0008472876836084e-06, 'adrastus': 2.7007625589152476e-05, 'neeks': 3.0008472876836084e-06, '203.1': 3.0008472876836084e-06, 'fencing': 3.0008472876836084e-06, 'marsh': 9.002541863050826e-06, 'bethoug

In [19]:
for filename, tf_idfs in tf_idf_non_universal.items():
    highest_tf_idfs = sorted(tf_idfs.items(), key=lambda x: x[1], reverse=True)[:5] 
    lowest_tf_idfs = sorted(tf_idfs.items(), key=lambda x: x[1], reverse=False)[:5] 

    print(f"\nHighest TF IDFs in {filename}")
    for word, tf_idf in highest_tf_idfs:
        print(f"{word}: {tf_idf}")

    print(f"\nLowest TF IDFs in {filename}")
    for word, tf_idf in lowest_tf_idfs:
        print(f"{word}: {tf_idf}")




Highest TF IDFs in tlg0012.tlg001.perseus-eng3.txt
hector: 0.0014464083926634993
athene: 0.0004891381078924281
drave: 0.00025507201945310673
honour: 0.00018905337912406733
valour: 0.0001830516845487001

Lowest TF IDFs in tlg0012.tlg001.perseus-eng3.txt
shallow: 3.0008472876836084e-06
pheneos: 3.0008472876836084e-06
burgeoneth: 3.0008472876836084e-06
quick-glancing: 3.0008472876836084e-06
wandereth: 3.0008472876836084e-06

Highest TF IDFs in tlg0012.tlg002.perseus-eng4.txt
eumaios: 0.0003155567157399831
alkinoos: 0.0002755565686743514
antinoos: 0.00025777872553407066
eurykleia: 0.00014666720590731608
eurymakhos: 0.00012444490198196517

Lowest TF IDFs in tlg0012.tlg002.perseus-eng4.txt
ridiculous: 4.444460785070184e-06
tyndarus: 4.444460785070184e-06
enabled: 4.444460785070184e-06
mixing-jugs: 4.444460785070184e-06
telemos: 4.444460785070184e-06

Highest TF IDFs in tlg0012.tlg001.perseus-eng4.txt
hektor: 0.0016421906136067444
therapôn: 0.00012684979687567754
pontos: 0.000116564678210082

TF-IDF is helpful in showing how important a specific word is in document relative to the entire corpus. If a token's TF_IDF score is high, it means that the term is frequently used in not many documents. If the TF-IDF score is low, it means the term either has a low frequency or it is frequent in all documents (like we saw with 'achilles' and 'odysseus'). 

I explored the non universal word set we created to look at the TF-IDF scores for words that only appear in one document. I listed the 5 highest and 5 lowest TF-IDF scores in each document. The high TF-IDF scores make some sense, like 'hector' being used very frequently in the first document and is thus important to the text. The low scores also make sense, with words like 'quick-glancing' that would not occur very often. Although, the scores also revealed some issues with the tokenization with tokens like '"' and 'us—', showing up, which would mess up the TF-IDF scores. In the future or if I had more time I would go back and fix up the tokenization to hopefully fix the issue. 

One thing I thought TF-IDF might be useful for would be for making something like a search engine. Like if someone searched a word, TF-IDF could be useful to help find the most relevant documents. 